In [ ]:
import gsapi
import numpy as np
import pandas as pd
import datetime as dt
import time
from datetime import datetime

### Reading data from a Google Sheets document using the Google Sheets API

In [ ]:
# load_data - the function of reading data from a Google Sheets document
def load_data(spreadsheet_id, sheet_and_range):
    data = gsapi.read_data(spreadsheet_id,sheet_and_range)
    return pd.DataFrame(data[1:], columns=data[0])

In [ ]:
spreadsheet_id = '1Ycg7zTxds9DZnDvTrFcyNNKuTUxg6Yy6WF0a8Wc02WQ' # document id
list_sheets = ['transactions!A:D', 'clients!A:C','managers!A:C', 'leads!A:F'] # range of loaded data from each page

In [ ]:
# reading the necessary data from the pages of the document
transactions = load_data(spreadsheet_id,list_sheets[0])
clients = load_data(spreadsheet_id,list_sheets[1])
managers = load_data(spreadsheet_id,list_sheets[2])
leads = load_data(spreadsheet_id,list_sheets[3])

### Cleaning and processing "transactions" data

In [ ]:
# clean - data cleaning function
def clean(data):
    for i in data.columns:
        data[i] = data[i].replace({'0001-01-01 00:00:00':np.nan})
        data[i] = data[i].replace({'00000000-0000-0000-0000-000000000000':np.nan})
        data[i] = data[i].replace({'':np.nan,'ig':'instagram','vkontakte':'vk',
                                  'ycard#!/tproduct/225696739-1498486363994':'ycard',
                                  'insta':'instagram'})

In [ ]:
clean(transactions)
transactions = transactions.rename(columns={"created_at":"tran_created_at", "l_client_id":"client_id"})
transactions['m_real_amount'] = transactions['m_real_amount'].astype(int)
transactions["tran_created_at"] = pd.to_datetime(transactions["tran_created_at"])

In [ ]:
transactions = transactions.drop_duplicates()

In [ ]:
print(transactions.shape)

In [ ]:
transactions.head()

### Cleaning and processing "clients" data

In [ ]:
clean(clients)
clients = clients.rename(columns={"created_at":"client_created_at", "l_manager_id":"manager_id"})
clients['client_created_at'] = pd.to_datetime(clients['client_created_at'])

In [ ]:
clients = clients.drop_duplicates()

In [ ]:
print(clients.shape)

In [ ]:
clients.head()

### Cleaning and processing “leads” data

In [ ]:
clean(leads)
leads = leads.rename(columns={"created_at":"lead_created_at", "l_manager_id":"manager_id", "l_client_id":"client_id"})
leads["lead_created_at"] = pd.to_datetime(leads["lead_created_at"])

In [ ]:
leads = leads.drop_duplicates()

In [ ]:
print(leads.shape)

In [ ]:
# adding columns "d_manager" and "d_club"
leads = leads.merge(managers, how='left', on='manager_id')

In [ ]:
print(leads.shape)
leads.head()

### Generating metrics for the final report

#### lead - number of orders

In [ ]:
leads['lead'] = 1

In [ ]:
print(leads.shape)
leads.head()

#### trash_lead - number of junk orders

In [ ]:
leads['trash_lead'] = leads.client_id.isna().astype('int')

In [ ]:
print(leads.shape)
leads.head(3)

#### new_lead - number of new orders (there were no orders and purchases from this client before)

In [ ]:
min_date = leads.lead_created_at.min()
print(min_date)

In [ ]:
# generating new customers based on purchase data
t = transactions[transactions.tran_created_at < min_date]
new_client_t = pd.merge(leads,t[['client_id','tran_created_at']],how='inner')
new_client_t['new_client_t'] = 1
new_client_t = new_client_t[['client_id','new_client_t']]

In [ ]:
# generating new clients based on data on previously registered clients
c = clients[clients.client_created_at < min_date]
new_client_c = pd.merge(leads,c[['client_id','client_created_at']],how='inner')
new_client_c['new_client_c'] = 1
new_client_c = new_client_c[['client_id','new_client_c']]

In [ ]:
leads = pd.merge(leads,new_client_t, how='left').drop_duplicates()
leads = pd.merge(leads,new_client_c, how='left').drop_duplicates()

In [ ]:
# creating the 'new_lead' column
new_client = ~(leads[['new_client_t','new_client_c']].sum(axis=1) > 0)
leads['new_lead'] = new_client.astype(int)
leads.drop(['new_client_t', 'new_client_c'], axis=1, inplace=True)

In [ ]:
print(leads.shape)
leads.head()

#### buyer - number of buyers (those who bought within a week after the application)

In [ ]:
# selecting new transactions where the purchase was made
pur = transactions[transactions.m_real_amount > 0]
pur = pur[pur.tran_created_at >= min_date]
print(pur.shape)
pur.head()

In [ ]:
# selecting transactions that were made after the application was submitted by the client
pur = pd.merge(pur,leads[['client_id','lead_created_at','lead_id']],how='inner')
pur = pur[pur.tran_created_at > pur.lead_created_at]
print(pur.shape)
pur.head()

In [ ]:
# selecting transactions that were made within a week after the application was submitted by the client
week = dt.timedelta(days=7)
pur = pur[(pur.tran_created_at - pur.lead_created_at) < week]
pur = pur.drop_duplicates(subset=['transaction_id','tran_created_at','m_real_amount'], keep='last')
print(pur.shape)
pur.head()

In [ ]:
# сreating the 'buyer' column
buyers = pur.groupby('client_id').m_real_amount.sum()
buyers = buyers.reset_index()
leads = pd.merge(leads, buyers, how='left')
leads['buyer'] = leads.m_real_amount.notna().astype(int)

In [ ]:
print(leads.shape)
leads.head()

#### new_buyer - number of new buyers (those who bought within a week after the application, and had not bought earlier)

In [ ]:
new_bayer = leads[['new_lead','buyer']].sum(axis=1)>1
leads['new_buyer'] = new_bayer.astype(int)

In [ ]:
print(leads.shape)
leads.head()

#### new_m_real_amount - income from new buyers’ purchases

In [ ]:
leads['new_m_real_amount'] = leads.new_buyer * leads.m_real_amount

In [ ]:
print(leads.shape)
leads.head()

In [ ]:
# filling missing values
leads = leads.fillna(0)
print(leads.shape)
leads.head()

### Final table and exporting data to a Google Sheets document using the Google Sheets API

In [ ]:
# converting dates to days
leads['lead_created_at'] = leads['lead_created_at'].dt.date
leads['lead_created_at'] = leads['lead_created_at'].astype(str) 
leads.head()

In [ ]:
dimensions = ['lead_created_at','d_utm_source','d_club','d_manager']
metrics = ['lead','trash_lead','new_lead','buyer','new_buyer','new_m_real_amount']

In [ ]:
df = leads[dimensions+metrics]
df = df.sort_values(by=dimensions)

In [ ]:
column_name = [df.columns.tolist()]
push_data = df.values.tolist()

In [ ]:
# document id
spreadsheet_id = '1xWas0rwKUqqll8M1GoMmwK8xByAKfoj4wmtscbQjI7g'

# range of exported data
sheet_range1 = 'dataframe!A1:J'
sheet_range2 = 'dataframe!A2:J'

In [ ]:
# exporting data to a Google Sheets document
gsapi.push_data(spreadsheet_id, sheet_range1, column_name)

In [ ]:
gsapi.push_data(spreadsheet_id, sheet_range2, push_data)

### Pivot table: 
https://docs.google.com/spreadsheets/d/1xWas0rwKUqqll8M1GoMmwK8xByAKfoj4wmtscbQjI7g/edit#gid=1859765491